In [18]:
import os
import pandas as pd
df = pd.read_csv('gossipcop_llm_rationales.csv')
current_path = os.getcwd()
print(current_path)
rationale_names = {'td','cs'}
df[:10]

/home/lyq/PycharmProjects/PycharmProjects


,content,label,image_id,source_id,split,td_rationale,td_pred,td_acc,cs_rationale,cs_pred,cs_acc
0,Congratulations might be in order for Miley Cy...,0,gossipcop-2493749932,gossipcop-2493749932,NaN,1. Source: The message is from a reputable so...,1,0,Plausibility: It is plausible that Miley Cyru...,1,0
1,Thousands are taking the streets to protest Pr...,0,gossipcop-941805037,gossipcop-941805037,NaN,- The message is about a protest against Pres...,1,0,Plausibility: It is plausible that people wou...,1,0
2,We'd venture to say that Cindy Crawford's daug...,0,gossipcop-2547891536,gossipcop-2547891536,NaN,- The message includes specific details about...,0,1,Plausibility: The message seems plausible as ...,1,0
3,"During the summer of 1995, John F. Kennedy Jr....",0,gossipcop-5189580095,gossipcop-5189580095,NaN,- The message is a fictional story about a me...,0,1,Plausibility: The description of the meeting ...,1,0
4,"Many stars saw great success in 2016, but the ...",0,gossipcop-9588339534,gossipcop-9588339534,NaN,- The message contains a list of specific cel...,1,0,Plausibility: The events and controversies li...,1,0
5,Caitlyn Jenner is trying to keep her romantic ...,0,gossipcop-8753274298,gossipcop-8753274298,NaN,"1. Source: The news article is from Variety, ...",1,0,Plausibility: The message seems plausible as ...,1,0
6,So while the internet is busy obsessing over J...,0,gossipcop-9878194459,gossipcop-9878194459,NaN,- The message contains specific details about...,1,0,Plausibility: The information provided seems ...,1,0
7,Next stop on the 2018 award season train? The ...,0,gossipcop-9521617242,gossipcop-9521617242,NaN,- Source: The message is from a news website ...,1,0,Plausibility: The message seems plausible as ...,1,0
8,"Angelina Jolie, 43, “really hasn’t come to ter...",0,gossipcop-2210288941,gossipcop-2210288941,NaN,"1. Source credibility: The source, HollywoodL...",1,0,Plausibility: It is plausible that Angelina J...,1,0
9,We should not be content to pass health care l...,0,gossipcop-3783917182,gossipcop-3783917182,NaN,1. Source: The message is from Senator Susan ...,1,0,Plausibility: The message is a political stat...,1,0


In [19]:
print(f"sum: {df.shape[0]}")
value_count = df['label'].value_counts()
print(f"real data: {value_count.get(1,0)}")
print(f"fake data: {value_count.get(0,0)}")

sum: 12332
real data: 9616
fake data: 2716


In [20]:
def filter_data(df):
    for r_name in rationale_names:
        df = df[ (df[f'{r_name}_pred'] != -1) & (df[f'{r_name}_rationale'] is not None) ]
    return df
        
print(f"before filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")
df = filter_data(df)
print(f"after filtering : {df.shape[0]}  real data :{(df['label']==1).sum()} fake data: {(df['label']==0).sum()}")


before filtering : 12332  real data :9616 fake data: 2716
after filtering : 12332  real data :9616 fake data: 2716


In [21]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score

for r_name in rationale_names:
    pred = df[r_name+ "_pred"].tolist()
    label = df['label'].tolist()
    print(f"{r_name}: acc {accuracy_score(label, pred)} recall_real: {recall_score(label, pred)} recall_fake: {recall_score(label, pred,pos_label=0)} precision_real: {precision_score(label, pred)} precision_fake: {precision_score(label, pred,pos_label=0)}")
    

td: acc 0.7814628608498216 recall_real: 0.9499792013311148 recall_fake: 0.18483063328424154 precision_real: 0.8049167327517843 precision_fake: 0.5106815869786369
cs: acc 0.7851929938371716 recall_real: 0.9595465890183028 recall_fake: 0.16789396170839468 precision_real: 0.8032558544441543 precision_fake: 0.5396449704142012


In [22]:
def split_data0(df,train_ratio=0.8,valid_ratio=0.1):
    train_nums = int(df.shape[0]*train_ratio)
    valid_nums = int(df.shape[0]*valid_ratio)
    return df.iloc[:train_nums],df.iloc[train_nums:train_nums+valid_nums],df.iloc[train_nums+valid_nums:]

def split_data(df,train_ratio=0.8,valid_ratio=0.1):
    real_data = df[df['label'] == 1]
    fake_data = df[df['label'] == 0]
    print(f"real_data count : {real_data.shape[0]}")
    print(f"fake_data count : {fake_data.shape[0]}")
    train_real,valid_real,test_real = split_data0(real_data,train_ratio,valid_ratio)
    train_fake,valid_fake,test_fake = split_data0(fake_data,train_ratio,valid_ratio)
    train_df,valid_df,test_df = pd.concat([train_real,train_fake]),pd.concat([valid_real,valid_fake]),pd.concat([test_real,test_fake])
    train_df['split'] = 'train'
    valid_df['split'] = 'val'
    test_df['split'] = 'test'
    return train_df,valid_df,test_df


train_df,valid_df,test_df = split_data(df,train_ratio=0.8,valid_ratio=0.1)


train_df.shape[0],valid_df.shape[0],test_df.shape[0]

real_data count : 9616
fake_data count : 2716


(9864, 1232, 1236)

In [23]:
train_df.to_csv('train.csv',index=False)
valid_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)